In [2]:
# import dependencies
import json
import pandas as pd
import os
import csv
import numpy
import requests
from pprint import pprint

#set up API URL
api_key = "a2rOTe8PfBiJBwTOSlRbteARqAEa0s6DsRyoSOOF"
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'

search = 'school.degrees_awarded.predominant=3&_fields=id,school.name,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,school.region_id,latest.earnings.10_yrs_after_entry.median,latest.earnings.6_yrs_after_entry.median,latest.repayment.5_yr_repayment.completers_rate,latest.repayment.7_yr_repayment.completers_rate,latest.repayment.3_yr_repayment.completers_rate'

search_url = url + search + '&api_key=' + api_key

response = requests.get(search_url)
response_json = response.json()
# pprint (response.json())


In [ ]:
# Store school id, name, regions in list

school_ids_list = []
schools_list = []
regions_list = []


for page in range (0,105):
    search_url = f'{url}{search}&page={page}&api_key={api_key}'
    response = requests.get(search_url)
    response_json = response.json()
       
    for num in range(0,20):
        school_id = response_json['results'][num]['id']
        school_ids_list.append(school_id)
        school_name = response_json['results'][num]['school.name']
        schools_list.append(school_name)  
        region_id = response_json['results'][num]['school.region_id']
        regions_list.append(region_id)

# test length of list to ensure equality
# print (len(school_ids_list))
# print (len(schools_list))
# print (len(regions_list))

# set up school info in dataframe and export to CSV to save work and avoid API limit
schoolinfo_df = pd.DataFrame({
    "School ID": school_ids_list,
    "School": schools_list,
    "Region ID": regions_list
})

schoolinfo_df.to_csv("schoolinfo.csv", index=False, header=False)
schoolinfo_df                    
            


In [ ]:
#store tuition information into lists
school_ids_list = []
tuition_in_state_list = []
tuition_out_state_list = []


for page in range (0,105):
        search_url = f'{url}{search}&page={page}&api_key={api_key}'
        response = requests.get(search_url)
        response_json = response.json()
        
        for num in range(0,20):
            school_id = response_json['results'][num]['id']
            school_ids_list.append(school_id)                
            tuition_in = response_json['results'][num]['latest.cost.tuition.in_state']
            tuition_in_state_list.append(tuition_in)  
            tuition_out = response_json['results'][num]['latest.cost.tuition.out_of_state']
            tuition_out_state_list.append(tuition_out)

#convert tuition lists into DataFrame and export
school_tuition_df = pd.DataFrame({
    "School ID": school_ids_list,
    "Tuition In of State": tuition_in_state_list,
    "Tuition Out of State": tuition_out_state_list
})
                
school_tuition_df.to_csv("school_tuition.csv", index=False, header=False)
school_tuition_df                

In [ ]:
#read csv paths as pandas and merge school info and tuition dataframes on school id
schoolcsvpath = os.path.join('schoolinfo.csv')
tuitioncsvpath = os.path.join('school_tuition.csv')

school_pd_df = pd.read_csv(schoolcsvpath, encoding="utf-8")
tuition_pd_df = pd.read_csv(tuitioncsvpath, encoding="utf-8")

school_tuition_merge = pd.merge(school_pd_df, tuition_pd_df, on='School Id', how="outer")

In [ ]:
#create lists for earnings, repayment rate

school_ids_list = []
list_earnings_10_yrs_after_entry = []
list_earnings_6_yrs_after_entry = []


for page in range (0,105):
    search_url = f'{url}{search}&page={page}&api_key={api_key}'
    response = requests.get(search_url)
    response_json = response.json()
    for num in range (0,20):
            earnings_10yrs_after = response_json['results'][num]['latest.earnings.10_yrs_after_entry.median']
            earnings_6yrs_after = response_json['results'][num]['latest.earnings.6_yrs_after_entry.median']
            list_earnings_10_yrs_after_entry.append(earnings_10yrs_after)
            list_earnings_6_yrs_after_entry.append(earnings_6yrs_after)

# len(list_earnings_10_yrs_after_entry)
# len(list_earnings_6_yrs_after_entry)

list_3_year_repayment = []
list_5_year_repayment = []
list_7_year_repayment = []


for page in range (0,105):
    search_url = f'{url}{search}&page={page}&api_key={api_key}'
    response = requests.get(search_url)
    response_json = response.json()
    for num in range (0,20):
            school_id = response_json['results'][num]['id']
            repay_3years = response_json['results'][num]['latest.repayment.3_yr_repayment.completers_rate']
            repay_5years = response_json['results'][num]['latest.repayment.5_yr_repayment.completers_rate']
            repay_7years = response_json['results'][num]['latest.repayment.7_yr_repayment.completers_rate']
            school_ids_list.append(school_id)
            list_3_year_repayment.append(repay_3years)
            list_5_year_repayment.append(repay_5years)
            list_7_year_repayment.append(repay_7years)

#convert lists into dataframes and export
earnings_repayment_df = pd.DataFrame({
    "School Id": school_ids_list,
    "Earnings 6 Years After Entry": list_earnings_6_yrs_after_entry,
    "Earnings 10 years After Entry": list_earnings_10_yrs_after_entry,
    "3 Year Repayment Percentage": list_3_year_repayment,
    "5 Year Repayment Percentage": list_5_year_repayment,
    "7 Year Repayment Percentage": list_7_year_repayment,
})
     

In [ ]:
#convert repayment/earnings info to csv and merge repayment/earnings information with school_tuition dataframe 

earnings_repayment_df.to_csv("earnings_repayment.csv", index=False, header=True)

earningscsvpath = os.path.join('earnings_repayment.csv')
earnings_pd_df = pd.read_csv(earningscsvpath, encoding="utf-8")


cost_earnings_stat_merge = pd.merge(school_tuition_merge, earnings_pd_df, on='School Id', how="outer")
cost_earnings_stat_merge.to_csv("cost_earnings_stat.csv", index=False, header=True)
cost_earnings_stat_merge


In [3]:
region_stats = pd.read_csv('cost_earnings_stat.csv')
region_stats

,School Id,School Name,Region Id,Tuition In State,Tuition Out of State,Earnings 6 Years After Entry,Earnings 10 years After Entry,3 Year Repayment Percentage,5 Year Repayment Percentage,7 Year Repayment Percentage
0,445188,University of California-Merced,8,13262.0,39944.0,36900.0,NaN,0.781473,0.786070,NaN
1,446792,Sanford-Brown College-Las Vegas,8,NaN,NaN,23500.0,29300.0,0.347468,0.444317,0.516055
2,446808,Sanford-Brown College-Seattle,8,NaN,NaN,23500.0,29300.0,0.347468,0.444317,0.516055
3,446048,Ave Maria University,5,19128.0,19128.0,29800.0,32800.0,0.674847,0.771186,0.711111
4,446668,The Art Institute of Ohio-Cincinnati,3,NaN,NaN,27200.0,34100.0,0.503601,0.603623,0.698751
5,447458,SAE Expression College,8,23899.0,23899.0,32100.0,48400.0,0.569519,0.609312,0.691803
6,447272,Argosy University-Los Angeles,8,13152.0,13152.0,30300.0,41400.0,0.476571,0.639640,0.761628
7,117557,Lincoln University,8,10575.0,10575.0,NaN,NaN,NaN,NaN,NaN
8,210492,Bryn Athyn College of the New Church,2,19932.0,19932.0,NaN,NaN,NaN,NaN,NaN
9,441982,Franklin W Olin College of Engineering,1,49986.0,49986.0,NaN,NaN,NaN,NaN,NaN


In [9]:
rs_dropna = region_stats.dropna()
rs_dropreset = rs_dropna.reset_index()

In [28]:
grouped_region_stats = rs_dropreset.groupby(['Region Id']).mean()
region_mean = grouped_region_stats.drop(columns=["index", "School Id"])
region_mean

,Tuition In State,Tuition Out of State,Earnings 6 Years After Entry,Earnings 10 years After Entry,3 Year Repayment Percentage,5 Year Repayment Percentage,7 Year Repayment Percentage
Region Id,,,,,,,
1,29112.816667,32776.183333,39475.000000,49554.166667,0.790872,0.839508,0.867417
2,26001.058608,29223.135531,37945.054945,48754.945055,0.739615,0.793213,0.828465
3,23448.936441,26742.444915,35246.186441,43145.338983,0.716357,0.779455,0.818114
4,20803.045161,23060.251613,35602.580645,42301.935484,0.761357,0.818104,0.844360
5,18366.325000,22730.782500,31787.000000,39603.750000,0.621347,0.691474,0.735573
6,16773.288889,21270.088889,34261.481481,41785.185185,0.644242,0.719577,0.762020
7,13179.042553,20291.914894,33821.276596,42572.340426,0.692003,0.764552,0.815170
8,23430.098684,28459.407895,36998.026316,47139.473684,0.698145,0.766504,0.814428
9,5910.650000,6726.500000,17500.000000,21735.000000,0.508256,0.548189,0.555709
